# Setup

In [1]:
import contracts
contracts.disable_all()

In [2]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

INFO:dt-world:duckietown-world 1.0.17
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class GenericData
DEBUG:dt-serialization:Registering class Sequence
DEBUG:dt-serialization:Registering class SampledSequence
DEBUG:dt-serialization:Registering class Constant
DEBUG:dt-serialization:Registering class RectangularArea
DEBUG:dt-serialization:Registering class TransformSequence
DEBUG:dt-serialization:Registering class VariableTransformSequence
DEBUG:dt-serialization:Registering class SE2Transform
DEBUG:dt-serialization:Registering class Scale2D
DEBUG:dt-serialization:Registering class Matrix2D
DEBUG:dt-serialization:Registering class SpatialRelation
DEBUG:dt-serialization:Registering class GroundTruth
DEBUG:dt-serialization:Registering class PlacedObject
DEBUG:dt-serialization:Registering class EvaluatedMetric
DEBUG:dt-serialization:Registering class GenericObject
DEBUG:dt-serialization:Registeri

In [3]:
dw.logger.setLevel(50)

Better visualization of output

In [4]:
%%html
<style>
pre {line-height: 90%}
</style>

# Road Network
Let's load a map and compute the road network.

In [5]:
m = dw.load_map('robotarium1')


Use the function `get_skeleton_graph`:

In [6]:
sk = dw.get_skeleton_graph(m)

The return type is `SkeletonGraphResult`. It contains in `sk.root2` a new map with the joined lane segments. 

In [7]:
ipython_draw_html(sk.root2);

While in the attribute `sk.G` we find a graph describing the topology.

This is a graph where each node is a meeting point between lanes, and each edge represents a lane.

In [23]:
# nodes
#print(len(list(sk.G.nodes())))
a = list(sk.G.nodes())
b = list(sk.G.edges())
# print(sk.G.graph)
# print(sk.G['P0'])
# print(sk.root2)

In [9]:
# Nodes as SE2
import geometry as geo
pos = {}
for n in sk.G:
    q = sk.G.nodes[n]['point'].as_SE2()
    t,p = geo.translation_angle_from_SE2(q)
    pos[n] = t
    # print t,p

In [10]:
# Access edges
for n1, n2 in sk.G.edges():
    data = sk.G.get_edge_data(n1, n2)
    one_lane = data[0]['lane']
    #print('I can go from %s to %s using lane %s' % (n1, n2, one_lane))

In [24]:
## Trying to extract all nodes in one direction
import numpy as np
from shapely.geometry import Polygon
import geometry as geo

## TODO: Experiment with these values
# duckie_length 
dl = 5.0
# duckie_width 
dw = 2.5

# Takes in 2 SE2 node objects, returns 1 if duckiebots
# at those locations simultaneously will cause a collision
# else returns 0
def overlap(n1, n2):
    p1, theta1 = geo.translation_angle_from_SE2(n1)
    p2, theta2 = geo.translation_angle_from_SE2(n2)
    poly1 = Polygon([((p1[0]+dw/2.0),(p1[1]+dl/2.0)), 
                     ((p1[0]-dw/2.0),(p1[1]+dl/2.0)), 
                     ((p1[0]-dw/2.0),(p1[1]-dl/2.0)),
                     ((p1[0]+dw/2.0),(p1[1]-dl/2.0))])
    poly2 = Polygon([((p2[0]+dw/2.0),(p2[1]+dl/2.0)), 
                     ((p2[0]-dw/2.0),(p2[1]+dl/2.0)), 
                     ((p2[0]-dw/2.0),(p2[1]-dl/2.0)),
                     ((p2[0]+dw/2.0),(p2[1]-dl/2.0))])
    return(1 if poly1.intersects(poly2) else 0) 
    
        
# Takes in empty array and populates with collision matrix values
def occupancyGrid(occupancy_grid):
    for ni,i in enumerate(sk.G):
        curr = sk.G.nodes[i]['point'].as_SE2()
        for nj,j in enumerate(sk.G):
            q = sk.G.nodes[j]['point'].as_SE2()
            #t,a = geo.translation_angle_from_SE2(q)
            occupied = overlap(curr,q)
            occupancy_grid[ni][nj] = occupied
    return occupancy_grid

# Iterate over all nodes and naively assigns them numerical values 
# corresponsing to their String names.
# Takes in list of nodes, Returns dict{'str': int} ex. dict{'P0': 0}
## TODO: This needs to be done only once!
def enumerateNodes(keys):
    node_labels = dict()
    for i, k in enumerate(keys):
        node_labels[k] = i
    return node_labels   

keys = list(sk.G.nodes())

# TODO: Instead have a dict {(P0,P1): bool}
occupancy_grid = np.zeros(shape=(len(keys),len(keys)))
occupancy_grid = occupancyGrid(occupancy_grid)

numerical_nodes = enumerateNodes(list(sk.G.nodes()))

# print occupancy_grid[1]
# print numerical_nodes


# Takes in a list of nodes (with numerical labels) and updates an 
# obstacle/occupied vector with a provided nodes "occupied' i.e. value=1
def spawnDuckiesNum(blockNode, total_nodes):
    node_num = np.zeros(shape=(1, total_nodes))
    for n in blockNode:
        node_num[n] = 1
    return node_num

# Takes in a list of nodes (with string labels) and updates 
# an obstacle/occupied vector with a provided nodes "occupied' i.e. value=1
def spawnDuckiesStr(blockNode, total_nodes):
    node_num = np.zeros(shape=(1, total_nodes))
    for n in blockNode:
        node_num[numerical_nodes[n]] = 1
    return node_num

"""
# Node diction: {node_name: [list of node neighbours]}
# Iterate over all nodes and find all potential neighbours
node_dict = dict()
for n1, n2 in sk.G.edges():
    data = sk.G.get_edge_data(n1, n2)
    one_lane = data[0]['lane']
    if n1 in list(node_dict.keys()):
        node_dict[n1].append(n2)
    else:
        node_dict[n1] = [n2]

#node list
keys = list(node_dict.keys())

for k in keys:
    neigh = node_dict[k]
    for n in neigh:
        # TODO: Send point as SE2
        occupied = overlap(k,n)
        occupancy_grid[(k,n)] = occupied
        print('occupied',k,n,occpied)
"""

"\n# Node diction: {node_name: [list of node neighbours]}\n# Iterate over all nodes and find all potential neighbours\nnode_dict = dict()\nfor n1, n2 in sk.G.edges():\n    data = sk.G.get_edge_data(n1, n2)\n    one_lane = data[0]['lane']\n    if n1 in list(node_dict.keys()):\n        node_dict[n1].append(n2)\n    else:\n        node_dict[n1] = [n2]\n\n#node list\nkeys = list(node_dict.keys())\n\nfor k in keys:\n    neigh = node_dict[k]\n    for n in neigh:\n        # TODO: Send point as SE2\n        occupied = overlap(k,n)\n        occupancy_grid[(k,n)] = occupied\n        print('occupied',k,n,occpied)\n"

Let's bring in the `draw_graph` function from some time ago.

In [ ]:
def draw_graph(G0, pos=None):
    import networkx as nx
    from matplotlib import pyplot as plt
    pos = pos or nx.spring_layout(G0)    
    plt.figure(figsize=(12, 12))    
    nx.draw(G0,pos,labels={node:node for node in G0.nodes()})
    def edge_label(a, b):
        datas = G0.get_edge_data(a, b)
        s = '%d edge%s' % (len(datas), 's' if len(datas)>=2 else '')
        for k, v in datas.items():
            if v:
                if 'label' in v:
                    s += '\n %s' % v['label']
                else:
                    s += '\n %s' %v
        return s
    #edge_labels = dict([ ((a,b), edge_label(a,b)) for a,b in G0.edges()])
    edge_labels = {}
    nx.draw_networkx_edge_labels(G0,pos,edge_labels=edge_labels,font_color='red')
    plt.axis('off')
    plt.show()

Set the position of each node in the graph based on the attribute 'point'.

In [ ]:
import geometry as geo
pos = {}
for n in sk.G:
    q = sk.G.nodes[n]['point'].as_SE2()
    t, _ = geo.translation_angle_from_SE2(q)
    pos[n] = t
draw_graph(sk.G, pos=pos)

## Planning example

Here is an example of how to do planning on the road network.

We select a start and end node:

In [ ]:
start = 'P60'
end = 'P36'

We find the shortest path:

In [ ]:
import networkx as nx
path = nx.shortest_path(sk.G, start, end)
print(path)

We retrieve the edge names:

In [ ]:
def get_lanes(path):
    edges = zip(path[:-1], path[1:]) 
    lanes  = []
    for a, b in edges:
        lane = sk.G.get_edge_data(a, b)[0]['lane']
        print ('a, b', a,b)
        print(sk.G.get_edge_data(a, b))
        lanes.append(lane)
    return lanes

In [ ]:
lanes = get_lanes(path);
print(lanes)

For visualization, we create a new map containing only the lanes selected: 

In [ ]:
po = dw.PlacedObject()
for lane_name in lanes:
    lane = sk.root2.children[lane_name]
    po.set_object(lane_name, lane, ground_truth=dw.SE2Transform.identity())

In [ ]:
ipython_draw_html(po);